## Setup

In [12]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pandas.plotting import register_matplotlib_converters
from sklearn.metrics import mean_squared_error
register_matplotlib_converters()

In [3]:
path = 'datasets/raw'
data = pd.read_parquet(f"{path}/input_dataset-2.parquet").reset_index()
predict_inputs = pd.read_parquet(f"{path}/prediction_input.parquet").reset_index()
data["timepoints"] = data["timepoints"].astype(np.int64)
predict_inputs["timepoints"] = predict_inputs["timepoints"].astype(np.int64)

## Models

First, define a linear normalizer. Since our models had issues learning the long term linear trend of the dataset, we instead normalize it with a linear function.

In [4]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline

class LinearNormalizer():
    def __init__(self,data):
        self.df = data
        nan_idx = ~np.isnan(data.iloc[:,1].values)
        X = data.iloc[:,0].values
        Y = data.iloc[:,9:15].values
        X = X[nan_idx].reshape(-1,1)
        Y = Y[nan_idx, :]
        regr = linear_model.LinearRegression()
        self.pipeline = Pipeline([("preprocess", StandardScaler()), ('poly', PolynomialFeatures(degree=1)), ("model", regr)])
        self.pipeline.fit(X, Y)

    def transform(self, X, Y):
        return Y - self.pipeline.predict(X)
    
    def inverse_transform(self, X, Y):
        return Y + self.pipeline.predict(X)


Then define main LSTM model

In [5]:
import torch #pytorch
import torch.nn as nn

class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, fully_connected_size):
        super(LSTM1, self).__init__()
        self.num_layers = num_layers #number of layers
        self.hidden_size = hidden_size #hidden state

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, fully_connected_size) #fully connected 1
        self.fc = nn.Linear(fully_connected_size, num_classes) #fully connected last layer

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #internal state

        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = output.reshape(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.fc(out) #Final Output
        return out

## Preprocessing of data

In [6]:
# One hot encoding of 'mode' and split values in X and Y
mode_one_hot = pd.get_dummies(data["mode"])
X = np.hstack([data.iloc[:,1:7].values, mode_one_hot.values])
Y_true = data.iloc[:,9:15].values

# Remove nans
nan_idx = ~np.isnan(X).any(axis=1)
X = X[nan_idx, :]
Y_true = Y_true[nan_idx, :]
N = X.shape[0]

In [7]:
# Normalize data with Linear Normalizer
linear_normalizer = LinearNormalizer(data)
X_time = data.iloc[:,0].values[nan_idx,None]
Y_normalized = linear_normalizer.transform(X_time, Y_true)

In [8]:
# Scale data to 0 mean unit variance
from sklearn.preprocessing import StandardScaler

x_scaler = StandardScaler().fit(X)
y_scaler = StandardScaler().fit(Y_normalized)

X = x_scaler.transform(X)
Y = y_scaler.transform(Y_normalized)

In [9]:
# Split data in training and test
train_split_fraction = 1

X_train = X[:int(N*train_split_fraction)]
Y_train = Y[:int(N*train_split_fraction)]

X_val = X[int(N*train_split_fraction):]
Y_val = Y[int(N*train_split_fraction):]

In [10]:
from torch.autograd import Variable 

seq_length = 100
X = X_train[:seq_length*(N//seq_length)]
Y = Y_train[:seq_length*(N//seq_length)]

# Reshape into (num_batches, seq_length, input_size)
X_tensor = Variable(torch.from_numpy(X).reshape(-1,seq_length,X.shape[1]).float())
Y_tensor = Variable(torch.from_numpy(Y).float())

## Training

Define training variables

In [11]:
num_epochs = 10 #1000 epochs
learning_rate = 0.001 #0.001 lr

input_size = 8 #number of features
hidden_size = 12 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

num_classes = 6 #number of output classes 

lstm1 = LSTM1(num_classes, input_size, hidden_size, num_layers, X_tensor.shape[1]) #our lstm class 
criterion = torch.nn.MSELoss()    # mean-squared error for regression

Main training loop, using LBFGS as optimizer

In [ ]:
optimizer = torch.optim.LBFGS(lstm1.parameters(), lr=0.08)
for epoch in range(num_epochs):
    def closure():
        optimizer.zero_grad()
        out = lstm1(X_tensor)
        loss = criterion(out, Y_tensor)
        loss.backward()
        print(f'Loss {loss.detach().item()}')
        return loss

    optimizer.step(closure) #improve from loss, i.e backprop
    with torch.no_grad():
        loss = closure()
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

torch.save(lstm1.state_dict(), "model.pth")

Loss 0.9891471266746521


## Plotting results and calculation of RMSE

In [ ]:
with torch.no_grad():
    lstm1.load_state_dict(torch.load("model.pth"))
    print("Model loaded")

    # Predicting test data and scaling back
    X_tensor = Variable(torch.from_numpy(X_train).reshape(1,-1,X_train.shape[1]).float())
    preds = lstm1(X_tensor).detach().numpy()
    preds = y_scaler.inverse_transform(preds)
    preds = linear_normalizer.inverse_transform(X_time[:int(N*train_split_fraction)], preds)

    # Calculate RMSE for train set
    err = mean_squared_error(Y_true[:int(N*train_split_fraction)], preds)
    print(err, np.sqrt(err))

    # Plotting predictions against training data
    for i in range(6):
        plt.plot(data["timepoints"][nan_idx][:int(N*train_split_fraction)], y_scaler.inverse_transform(Y_true[:int(N*train_split_fraction)])[:,i])
        plt.plot(data["timepoints"][nan_idx][:int(N*train_split_fraction)], y_scaler.inverse_transform(preds)[:,i])
        plt.show()

    # Predicting train data and scaling back
    X_val_tensor = Variable(torch.from_numpy(X_val).reshape(1,-1,X_val.shape[1]).float())
    preds = lstm1(X_val_tensor).detach().numpy()
    preds = y_scaler.inverse_transform(preds)
    preds = linear_normalizer.inverse_transform(X_time[int(N*train_split_fraction):], preds)

    # Calculate RMSE test set
    err = mean_squared_error(Y_true[int(N*train_split_fraction):], preds)
    print(err, np.sqrt(err))

    # Plotting predictions against test data
    for i in range(6):
        plt.plot(data["timepoints"][nan_idx][int(N*train_split_fraction):], y_scaler.inverse_transform(Y_true[int(N*train_split_fraction):])[:,i])
        plt.plot(data["timepoints"][nan_idx][int(N*train_split_fraction):], y_scaler.inverse_transform(preds)[:,i])
        plt.show()

Make csv for delivery

In [ ]:
with torch.no_grad():
    predict_inputs = pd.read_parquet("datasets/raw/prediction_input.parquet").reset_index()
    df = pd.DataFrame({"timepoints": predict_inputs["timepoints"]})
    
    mode = pd.get_dummies(predict_inputs["mode"])
    predict_inputs["timepoints"] = predict_inputs["timepoints"].astype(np.int64)
    X_test = np.hstack([predict_inputs.iloc[:,1:7].values, mode.values])
    X_test = x_scaler.transform(X_test)
    X_test_tensor = Variable(torch.from_numpy(X_test).reshape(1,X_test.shape[0],X_test.shape[1]).float())

    X_test_time = predict_inputs.iloc[:,0].values[:,None]

    preds = lstm1(X_test_tensor).detach().numpy()
    preds = y_scaler.inverse_transform(preds)
    preds = linear_normalizer.inverse_transform(X_test_time, preds)

    for i in range(6):
        plt.plot(preds[:,i])
        plt.show()

    for i in range(6):
        df[f"Bolt_{i+1}_Tensile"] = preds[:,i]

    df.to_csv("predict_result.csv", index=False)
